Instalación de la libreia deap, si requiere mayor información consultar:

https://deap.readthedocs.io/en/master/

Integrantes:
Adrian Felipe Gironza - Arlex Fabian Galindez


In [ ]:
!pip install deap

In [50]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import numpy as np
import json
from collections import Counter
import copy

In [51]:
np.set_printoptions(suppress=True)

def cargar_datos_asignaturas():
    # Cargar los datos de asignaturas desde un archivo JSON
    with open("asignaturas.json", encoding="utf-8-sig") as file:
        datos = json.load(file)
    return datos

def cargar_datos_salones():
    # Cargar los datos de salones desde un archivo JSON
    with open("salones.json", encoding="utf-8-sig") as file:
        datos_sal = json.load(file)
    return datos_sal

datos_asignaturas = cargar_datos_asignaturas()
datos_salones = cargar_datos_salones()

cantasignaturas = len(datos_asignaturas)
cantdias = 5
cantfranjas=5
cantsalones= len(datos_salones)
valorlibre = 0

In [52]:
def initIndividual(icls):

    ind = iniciarmatriz(cantdias,cantfranjas,cantsalones)
    ind = asignarmaterias(ind)
    ind = corregir_asignaturas_repetidas(ind)
    ind = corregir_profesores_repetidos(ind)
    ind = corregir_semestres_repetidos(ind)
    ind = corregir_asignaturas_por_dia(ind)
    return icls(ind)

def iniciarmatriz(x, y, z):
    matriz = np.zeros((x, y, z),dtype=np.int)
    valor_predeterminado = valorlibre
    for i in range(x):
        for j in range(y):
            for k in range(z):
                matriz[i, j, k] = valor_predeterminado
    return matriz

def asignarmaterias(ind):
    lind = ind.ravel()
    for asignatura in range(cantasignaturas):
        jornasignadas = datos_asignaturas[asignatura]["JORNEC"]
        for j in range(jornasignadas):  # Utilizamos _ para indicar que no se usa la variable de iteración
            while True:
                posicion_aleatoria = random.randint(0, len(lind) - 1)
                if lind[posicion_aleatoria] == valorlibre:
                    lind[posicion_aleatoria] = asignatura
                    break
    nind = lind.reshape(cantdias, cantfranjas, cantsalones)
    return nind

In [53]:
def corregir_asignaturas_repetidas(individuo):
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            salones = individuo[dia, franja]  # Obtener los salones asignados en la franja horaria
            asignaturas = list(set(salones))  # Obtener las asignaturas únicas en la franja
            if len(asignaturas) < len(salones):
                for asignatura in asignaturas:
                    indices = np.where(salones == asignatura)[0]  # Obtener los índices de la asignatura
                    if len(indices) > 1:
                        for i in range(1, len(indices)):
                            posicion = indices[i]
                            if salones[posicion] != valorlibre:
                                # Cambiar la asignatura de franja
                                nueva_franja = (franja + i) % cantfranjas
                                nueva_salon = np.where(individuo[dia, nueva_franja] == valorlibre)[0][0]
                                individuo[dia, nueva_franja, nueva_salon] = salones[posicion]
                                salones[posicion] = valorlibre  # Marcar la asignatura como libre en la franja original
    return individuo

def corregir_profesores_repetidos(individuo):
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            for salon in range(cantsalones):  # Iterar sobre los salones en la franja horaria
                asignatura = (individuo[dia, franja, salon])
                if asignatura != valorlibre:
                    profesor = datos_asignaturas[asignatura]['PROFESOR']
                    # Verificar si hay más de una asignatura asignada al mismo profesor en la franja horaria
                    otras_asignaturas = [individuo[dia, franja, s] for s in range(cantsalones) if s != salon]
                    if otras_asignaturas.count(asignatura) > 0:
                        nueva_franja = (franja + 1) % cantfranjas  # Obtener la siguiente franja horaria
                        nueva_salon = np.where(individuo[dia, nueva_franja] == valorlibre)[0][0]
                        individuo[dia, nueva_franja, nueva_salon] = asignatura
                        individuo[dia, franja, salon] = valorlibre  # Marcar la asignatura como libre en la franja original
    return individuo

def corregir_semestres_repetidos(individuo):
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            asignaturas = individuo[dia, franja]  # Obtener las asignaturas de la franja
            semestres = set()
            for asignatura in asignaturas:
                if asignatura != valorlibre:  # Ignorar las asignaturas no asignadas
                    semestre = datos_asignaturas[asignatura]['ASIGNATURA']
                    if semestre in semestres:
                        # Encontró asignatura repetida del mismo semestre
                        indices = np.where(asignaturas == asignatura)[0]
                        for i in range(1, len(indices)):
                            posicion = indices[i]
                            if asignaturas[posicion] != valorlibre:
                                # Buscar otro salón aleatorio libre en otra franja
                                nueva_franja = (franja + i) % cantfranjas
                                nueva_salon = np.where(individuo[dia, nueva_franja] == valorlibre)[0][0]
                                individuo[dia, nueva_franja, nueva_salon] = asignaturas[posicion]
                                asignaturas[posicion] = valorlibre  # Marcar la asignatura como libre en la franja original
                    semestres.add(semestre)
    return individuo

def corregir_asignaturas_por_dia(individuo):
    for dia in range(cantdias):
        asignaturas_dia = individuo[dia]  # Obtener las asignaturas del día
        asignaturas_unicas = np.unique(asignaturas_dia)  # Obtener las asignaturas únicas
        if len(asignaturas_unicas) < len(asignaturas_dia):
            asignaturas_asignadas = set()
            for i in range(cantfranjas):
                for j in range(cantsalones):
                    asignatura = asignaturas_dia[i, j]
                    if asignatura != valorlibre:
                        if asignatura in asignaturas_asignadas:
                            # Buscar otra franja y salón aleatorio que esté libre y no tenga asignatura duplicada
                            nueva_franja = random.randint(0, cantfranjas - 1)
                            nueva_salon = random.randint(0, cantsalones - 1)
                            while individuo[dia, nueva_franja, nueva_salon] != valorlibre or individuo[dia, nueva_franja, nueva_salon] in asignaturas_asignadas:
                                nueva_franja = random.randint(0, cantfranjas - 1)
                                nueva_salon = random.randint(0, cantsalones - 1)
                            # Realizar el intercambio de asignaturas
                            individuo[dia, i, j] = individuo[dia, nueva_franja, nueva_salon]
                            individuo[dia, nueva_franja, nueva_salon] = asignatura
                        else:
                            asignaturas_asignadas.add(asignatura)
    return individuo


In [54]:
def validar_profesor_nn(individuo):
    contador = 0
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            asignaturas = individuo[dia, franja, :] # Obtener las asignaturas de la franja
            profesores = [datos_asignaturas[asignatura]["PROFESOR"] for asignatura in asignaturas]
            contador_nn = sum(1 for p in profesores if p in ["NN3", "Becario", "Asistente de Docencia"])
            if contador_nn > 1:
                contador += 1
    return contador

def validar_franjas_asignatura(individuo):
    contador_repetidas = 0
    for dia in range(cantdias):
        for asignatura in range(cantasignaturas):
            franjas_dia = individuo[dia, :, :].flatten()  # Obtener todas las franjas del día
            frecuencias = Counter(franjas_dia)
            asignatura_repetida = [frecuencia for _, frecuencia in frecuencias.items() if frecuencia > 1]
            if asignatura_repetida:
                contador_repetidas += 1
    return contador_repetidas

def validar_tipo_asignatura(individuo):
    contador = 0
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            for salon in range(cantsalones):
                codigo_asignatura = individuo[dia, franja, salon]
                asignatura = datos_asignaturas[codigo_asignatura]
                tipo_asignatura = asignatura['TIPOAS']
                tipo_salon = datos_salones[salon]['TIPO']
                if tipo_asignatura == 'T' and tipo_salon != 'TEORICO':
                    contador += 1
                if tipo_asignatura == 'P' and tipo_salon != 'LAB':
                    contador += 1
                if tipo_asignatura == 'TP' and tipo_salon != 'LAB':
                    contador += 1
                if tipo_asignatura == 'P' and int(asignatura['ASIGNATURA']) >= 7 and datos_salones[salon]['CODIGO'] != 'Sala 4':
                    contador += 1
    return contador

In [55]:
def evalOneMax(ind):
    fitness = 0
    fitness += validar_profesor_nn(ind)
    fitness += validar_franjas_asignatura(ind)
    fitness += validar_tipo_asignatura(ind)
    return (fitness,)

In [56]:

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual_guess)

In [57]:
def cxTwoPointCopy(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoints = sorted(random.sample(range(1, size), 2))
    for i in range(len(cxpoints)):
        cxpoint1, cxpoint2 = cxpoints[i-1], cxpoints[i]
        temp_ind1 = ind1[cxpoint1:cxpoint2].copy()
        temp_ind2 = ind2[cxpoint1:cxpoint2].copy()

        for dia in range(cantdias):
            for franja in range(cantfranjas):
                for salon in range(cantsalones):
                    if ind1[dia, franja, salon] in temp_ind2:
                        ind1[dia, franja, salon] = temp_ind2[temp_ind2 == ind1[dia, franja, salon]][0]
                    if ind2[dia, franja, salon] in temp_ind1:
                        ind2[dia, franja, salon] = temp_ind1[temp_ind1 == ind2[dia, franja, salon]][0]

    return ind1, ind2

def cxOnePointCopy(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoint = random.randint(1, size-1)
    ind1[cxpoint:], ind2[cxpoint:] = copy.deepcopy(ind2[cxpoint:]), copy.deepcopy(ind1[cxpoint:])
    return ind1, ind2

def mutFlipBit(individual, indpb):
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            for salon in range(cantsalones):
                if random.random() < indpb:
                    individual[dia, franja, salon] = random.randint(0, cantasignaturas-1)
    return individual,

In [58]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

In [59]:
def main():
    random.seed(64)

    pop = toolbox.population(n=2)
    hof = tools.HallOfFame(1, similar=np.array_equal)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5,mutpb=0.1, ngen=10, stats=stats,halloffame=hof)

    best_individual = hof[0]
    print('Mejor solución:', best_individual)

    horario = decode_individual(best_individual)
    print('Horario:')
    print_horario(horario)

    return pop, stats, hof

def decode_individual(individual):
    horario = np.zeros((cantdias, cantfranjas, cantsalones), dtype=object)
    for dia in range(cantdias):
        for franja in range(cantfranjas):
            for salon in range(cantsalones):
                codigo_asignatura = individual[dia, franja, salon]
                if codigo_asignatura != valorlibre:
                    asignatura = datos_asignaturas[codigo_asignatura]
                    nombre_asignatura = asignatura['NOMBRE']
                    profesor = asignatura['PROFESOR']

                    codigo_salon = datos_salones[salon]['CODIGO']  # Obtener código del salón

                    franja_horaria = f'{franja}-{franja+1}'
                    horario[dia, franja, salon] = (nombre_asignatura, profesor, franja_horaria, codigo_salon)
    return horario


def print_horario(horario):
    for dia in range(cantdias):
        print(f'Día {dia+1}:')
        for franja in range(cantfranjas):
            inicio_franja = 7 + franja * 2
            fin_franja = inicio_franja + 2
            if fin_franja > 13:
                inicio_franja += 1
                fin_franja += 1
            print(f'Franja horaria {inicio_franja}-{fin_franja}:')
            for salon in range(cantsalones):
                asignatura = horario[dia, franja, salon]
                if asignatura:
                    nombre_asignatura, profesor, franja_horaria, codigo_salon = asignatura
                    print(f'Salón {codigo_salon}: {nombre_asignatura} - Profesor: {profesor}')
            print()
        print()

if __name__ == "__main__":
    p, s, h = main()

<ipython-input-52-8264f7a4b4a3>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  matriz = np.zeros((x, y, z),dtype=np.int)


gen	nevals	avg	std	min
0  	2     	894	2  	892
1  	2     	892	0  	892
2  	2     	892	0  	892
3  	0     	892	0  	892
4  	0     	892	0  	892
5  	2     	892	0  	892
6  	2     	892	0  	892
7  	2     	892	0  	892
8  	0     	892	0  	892
9  	0     	892	0  	892
10 	2     	892	0  	892
Mejor solución: [[[  0  86   0   2  16   0   0  23  90   0   0  30  39   5  26 100]
  [  0  15   0  34   0   0  96   0   0   0  88   0   0  62   0  27]
  [ 45   0   0   0   0  32   0  85   0   0   0   0   0  87   0   0]
  [ 73   0   0   0   0   0   0  86   0   0   0   0   0   0   0  91]
  [  0  92  81  41  97   4  10   1   0   0  47  36   0   0   0   0]]

 [[  0  99  93   0  19  60  31   0  14   0   0  87   0  67   0  15]
  [  0   0   0   0   0   0   0   0  11  42  12   0   0   0   0  98]
  [  0   0   0   0  96   0   0  49   0  44  94   0   0  52   0   0]
  [ 29   0   0   0  13   0   0   0   0   0  55  85  40   7  24  22]
  [  0   2  51   0  21   0   0   0   0   0   0   0  63   0  23   3]]

 [[  6  38   0   0   0  